In [94]:
import numpy as np
from fish.util.fileio import read_image, read_images
import matplotlib.pyplot as plt
from glob import glob
from fish.util.plot import proj_fuse
from fish.image.vol import get_stack_freq
from skimage.transform import downscale_local_mean

#stack_freq,_,_ = get_stack_freq('/groups/ahrens/ahrenslab/YuMu/SPIM/active_datasets/20171229/fish1/20171229_1_5_hucrgeco_gfapcochr_7dpf_stimwaist_imagedwiithhigherpower_20171229_135523/raw/')
stack_freq,_,_ = get_stack_freq('/groups/ahrens/ahrenslab/davis/data/spim/raw/20180208/6dpf_cy252xcy293_optostim_test_20180208_185317/')
um_per_pix = .40625
# this should be inferred from metadata...
ds_xy = 2
%matplotlib inline

In [2]:
from functools import reduce
from math import gcd
aspect = np.array((8, ds_xy * um_per_pix, ds_xy * um_per_pix))
aspect *= 10
aspect //= reduce(gcd, np.round(aspect).astype('int'))
aspect=aspect.astype('int')

In [ ]:
from fish.image.vol import unfilter_flat
import pathlib as pl
from os.path import sep
condition_dirs = sorted(glob('/groups/ahrens/ahrenslab/davis/data/spim/proc/20180208/6dpf_cy252xcy293_optostim_test_20180208_185317/opto/condition_*/'))
av_window = np.load('/groups/ahrens/ahrenslab/davis/data/spim/proc/20180208/6dpf_cy252xcy293_optostim_test_20180208_185317/opto/av_window.npy')
mask = read_image('/groups/ahrens/ahrenslab/davis/data/spim/proc/20180208/6dpf_cy252xcy293_optostim_test_20180208_185317/opto/mask.tif')
out_path = str(pl.Path(condition_dirs[0]).parent) + sep
mask = mask[:,::ds_xy,::ds_xy]
tr_len = len(av_window)

In [210]:
for ind,c_dir in enumerate(condition_dirs):
    fnames = sorted(glob(c_dir + '*.npy'))
    num_trials = len(fnames) // tr_len
    data = np.array([np.load(f) for f in fnames])
    data_trial = data.reshape(num_trials, tr_len, data.shape[1])
    # get the max across (x,y) for each plane to find timepoints where the laser bled through
    mx_by_plane = np.array([[unfilter_flat(d, mask).max((1,2)) for d in tr] for tr in data_trial])
    # mask for masked array that blanks timepoints where the max value was too high
    thr = .35
    valid_mask = mx_by_plane - np.median(mx_by_plane, axis=0)
    result = np.zeros([data_trial.shape[1], *mask.shape])

    # loop through timepoints, averaging along valid trials 
    for t in range(data_trial.shape[1]):
        trial_vol = np.array([unfilter_flat(dt, mask) for dt in data_trial[:,t]])
        full_mask = np.zeros(trial_vol.shape[::-1])
        full_mask[:,:] = valid_mask[:,t].T
        full_mask = full_mask.T
        result[t] = np.ma.array(trial_vol, mask=full_mask > thr).mean(0)

    from skimage.io import imsave
    imsave(out_path + 'condition_{0}_mean.tif'.format(ind+1), result.astype('float32'), imagej=True)

/groups/ahrens/home/bennettd/anaconda3/lib/python3.6/site-packages/skimage/external/tifffile/tifffile.py:1088: UserWarning: truncating ImageJ file
  warnings.warn("truncating ImageJ file")


In [ ]:
stim_dur = 3.0
onset = np.where((av_window / stack_freq) == 0)[0][0]
offset = np.where((av_window / stack_freq) > stim_dur)[0][0]

In [ ]:
plt.imshow(proj_fuse(np.nanmax(data_c[:onset], axis=0), np.nanmax, aspect=aspect), origin='lower')

In [ ]:
fig, axs = plt.subplots(nrows=3, ncols=1)
fig.set_facecolor('black')
axs[0].imshow(proj_fuse(np.nanmean(data_c[offset:], axis=0) - np.nanmean(data_c[:onset], axis=0), np.nanmax, aspect=aspect), origin='lower')

In [ ]:
plt.imshow(proj_fuse(np.nanmean(data_c[offset:], axis=0), np.nanmin, aspect=aspect), origin='lower')

In [ ]:
from skimage.io import imread, imsave
from skimage.external import tifffile as tf
test_data = np.random.randn(10,10,10,10).astype('float32')
imsave('/groups/ahrens/ahrenslab/davis/test.tif', test_data, imagej=True, metadata={'axes' : 'ZYX', 'resolution_zyx' : [1,.1,.1]})
import ast
ast.literal_eval(b.as_str()[2:-1])['resolution_zyx']

In [ ]:
a = tf.TiffFile('/groups/ahrens/ahrenslab/davis/test.tif')

In [ ]:
data = read_images(fnames)

In [ ]:
# apply a median filter in time to deal with funky missing data
from scipy.ndimage.filters import median_filter
data_mf = median_filter(data, (3,1,1,1))

In [ ]:
data_c = data_mf.copy()
data_c[data_c == 0] = np.NAN

In [ ]:
data.shape
imsave('/groups/ahrens/ahrenslab/davis/test.tif', data_mf, imagej=True)

In [ ]:
data_mf.shape

In [ ]:
num_plots = data_mf.shape[0] // 5
slices = [np.arange(num_plots) + t for t in range(0, data_mf.shape[0], num_plots)]

In [ ]:
num_plots = data_mf.shape[0] // 5
slices = [np.arange(num_plots) + t for t in range(0, data_mf.shape[0], num_plots)]
nr = 2
nc = num_plots
fig, axs = plt.subplots(figsize=(8 * nc,8 * nr), nrows=nr, ncols=nc)
fig.set_facecolor('black')
for ind, sl in enumerate(slices):
    axs[0][ind].imshow(proj_fuse(np.nanmax(data_c[sl],axis=0), np.nanmax, aspect=aspect.astype('int')).T[:-260], cmap='gray', clim=[0,1])
    axs[1][ind].imshow(proj_fuse(np.nanmin(data_c[sl],axis=0), np.nanmin, aspect=aspect.astype('int')).T[:-260], cmap='gray', clim=[-.1,.4])    
plt.subplots_adjust(wspace=-.6,hspace=0)
[ax.axis('off') for ax in axs.ravel()];